Author: Madhusudhanan Balasubramanian (MB), Ph.D., The University of Memphis

June 21, 2024:
    * Copy of Copy22-3 (20240616_1)
    * PRIOR_PROB: 0.005 (from 0.01)
    July 12, 2024:
        * In Inspect_AP_vs_TH_segm.ipynb, AP/AR is not affected by the INFERENCE_TH_TEST values.  Checking visually

June 15, 2024:
    * Copy22-2 copy of Copy22-1
    * BATCH_SIZE_PER_IMAGE: 600
    * POSITIVE FRACTION: 0.5
    * Initial weights: VovNet weights

June 13, 2024:
    * Copy22-1 copy of Copy20-1
    * MIN_SIZE_TRAIN = [1024]
    * MIN_SIZE_TEST = [1024]
    * SIZES_OF_INTEREST = [20, 24, 28, 36]

June 05, 2024:
    * Copy21-1 copy of Copy20-1
    * MIN_SIZE_TRAIN = [512, 768, 1024]
    
April 19, 2024: Copy18-1 copy of Copy17-2:
    * Initialize with model_final_10Oct2024.pth - does this retain the higher AP on both training & validation for Necrotic & Live classes?

April 04, 2024: Copy17-2 copy of Copy17-1:
    * Benefit of higher warmup period or lowering the LR?
    * Higher FCOS.INFERENCE_TH_TEST & TRAIN 0.05 to 0.7
    * Fixed figures: utils/axon_visualizer.py & axon_colormap.py; evaluation/axon_evaluation.py & axon_evaluator.py
    
April 04, 2024: Copy of 16 - testing NUM_CLASSES = 2

April 03, 2024: Copy of 16 - adjusting parameters as in Copy4

Mar 28, 2024: Copy of 15 - running for same iterations with 3 classes instead of 2

Mar 04, 2024: Copy 12 copy of Copy 11 - refining weights with max_dets_per_image parameter for COCOEvaluator in the Trainer class (27Jan2024.pth weights gave lowers performance)

Jan 28, 2024: Copy 11 copy of Copy 10 - more simplified, running with Detectron2 ver. 0.6 (able to specify max_dets_per_image during evaluation)

Jan 27, 2024: Copy10 copy of Copy 9 - rerunning to establish a starting point

Nov 30, 2023: Copy9 copied from Copy7 - modified to resume training from the final Oct 10 model "model_final_10Oct2023.pth"

In [ ]:
# Work around to enhance error display involving unicode characters
%colors nocolor
%xmode plain

# import various libraries
import logging
import os, re
import numpy as np
import cv2
import matplotlib.pyplot as plt
#
import torch
#
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances, load_coco_json
#
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.engine import default_argument_parser, default_setup, hooks, launch
#
#MB: 03/31/2024
#from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from centermask.evaluation import COCOEvaluator, inference_on_dataset
#
#from detectron2.utils.visualizer import Visualizer, ColorMode
from centermask.utils.axon_visualizer import Visualizer, ColorMode
#
from centermask.config import get_cfg

#Trainer class
class Trainer(DefaultTrainer):
    """
    This is the same Trainer except that we rewrite the
    `build_train_loader` method.
    """

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        """
        Create evaluator(s) for a given dataset.
        This uses the special metadata "evaluator_type" associated with each builtin dataset.
        For your own dataset, you can simply create an evaluator manually in your
        script and do not have to worry about the hacky if-else logic here.
        """
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        evaluator_list = []
        evaluator_list.append(COCOEvaluator(dataset_name, output_dir=output_folder, max_dets_per_image=600))
        return evaluator_list[0]

#Configuration    
cfg = get_cfg()
cfg.merge_from_file("configs/AxonClass/axon_V_39_eSE_FPN_ms_3x.yaml")
cfg.MODEL.WEIGHTS = "model_819_2_0025999.pth"
cfg.num_gpus = 1
cfg.resume = True
cfg.freeze()
default_setup(cfg, '')

#Register training axon datasets
name = "Phase3_Training"
json_file = "./datasets/Phase3_Training/Phase3_Training.json"
image_root = "./datasets/Phase3_Training"
if name not in DatasetCatalog.list():
    register_coco_instances(name, {}, json_file, image_root)

Phase3_Tr_metadata = MetadataCatalog.get("Phase3_Training")
print(f"Phase3_Tr_metadata:{Phase3_Tr_metadata}")
Phase3_Tr_dicts = DatasetCatalog.get("Phase3_Training")

#Register validation axon datasets
name = "Phase3_Validation"
json_file = "./datasets/Phase3_Validation/Phase3_Validation.json"
image_root = "./datasets/Phase3_Validation"
if name not in DatasetCatalog.list():
    register_coco_instances(name, {}, json_file, image_root)

Exception reporting mode: Plain
[08/23 17:58:57 detectron2]: Rank of current process: 0. World size: 1
[08/23 17:58:58 detectron2]: Environment info:
----------------------  ---------------------------------------------------------------------------------------------------
sys.platform            linux
Python                  3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:21) [GCC 9.4.0]
numpy                   1.21.6
detectron2              0.6 @/opt/anaconda3/envs/venv_mamba/envs/venv_py37_D2_6a/lib/python3.7/site-packages/detectron2
Compiler                GCC 7.3
CUDA compiler           CUDA 11.3
detectron2 arch flags   3.7, 5.0, 5.2, 6.0, 6.1, 7.0, 7.5, 8.0, 8.6
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.10.0 @/opt/anaconda3/envs/venv_mamba/envs/venv_py37_D2_6a/lib/python3.7/site-packages/torch
PyTorch debug build     False
GPU available           Yes
GPU 0,1                 Quadro RTX 8000 (arch=7.5)
Driver version          460.32.03
CUDA_HOME   

Training

In [ ]:
trainer = Trainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()

FPN -> in_features: ['stage2', 'stage3', 'stage4']
FPN -> strides: [4, 8, 16]
FPN -> top_block: None
[08/23 17:59:04 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bottom_up): VoVNet(
      (stem): Sequential(
        (stem_1/conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (stem_1/norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        (stem_1/relu): ReLU(inplace=True)
        (stem_2/conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding

/opt/anaconda3/envs/venv_mamba/envs/venv_py37_D2_6a/lib/python3.7/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/opt/anaconda3/envs/venv_mamba/envs/venv_py37_D2_6a/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272168290/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[08/23 17:59:30 d2.utils.events]:  eta: 1 day, 8:08:51  iter: 10019  total_loss: 0.7111  loss_mask: 0.07818  loss_maskiou: 0.0002805  loss_fcos_cls: 0.0004636  loss_fcos_loc: 0.04529  loss_fcos_ctr: 0.587  time: 1.2376  data_time: 0.0215  lr: 0.01  max_mem: 11783M
[08/23 17:59:54 d2.utils.events]:  eta: 1 day, 7:13:04  iter: 10039  total_loss: 0.7048  loss_mask: 0.07256  loss_maskiou: 0.0002527  loss_fcos_cls: 0.0003658  loss_fcos_loc: 0.0437  loss_fcos_ctr: 0.587  time: 1.2212  data_time: 0.0083  lr: 0.01  max_mem: 11783M
[08/23 18:00:18 d2.utils.events]:  eta: 1 day, 7:08:31  iter: 10059  total_loss: 0.7026  loss_mask: 0.07213  loss_maskiou: 0.0002425  loss_fcos_cls: 0.0003254  loss_fcos_loc: 0.04411  loss_fcos_ctr: 0.5861  time: 1.2206  data_time: 0.0076  lr: 0.01  max_mem: 11783M
[08/23 18:00:44 d2.utils.events]:  eta: 1 day, 8:03:08  iter: 10079  total_loss: 0.7027  loss_mask: 0.07348  loss_maskiou: 0.0002305  loss_fcos_cls: 0.0003376  loss_fcos_loc: 0.04256  loss_fcos_ctr: 0.5873

No predictions from the model!


MB - Inside COCOEvaluator.evaluate(img_ids=None)
MB - Inside COCOEvaluator._eval_predictions
[08/24 05:08:57 d2.engine.defaults]: Evaluation results for Phase3_Validation in csv format:
[08/24 05:08:57 d2.evaluation.testing]: copypaste: Task: bbox
[08/24 05:08:57 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[08/24 05:08:57 d2.evaluation.testing]: copypaste: nan,nan,nan,nan,nan,nan
[08/24 05:08:57 d2.utils.events]:  eta: 21:17:25  iter: 40999  total_loss: 0.6884  loss_mask: 0.06407  loss_maskiou: 0.00026  loss_fcos_cls: 0.0004569  loss_fcos_loc: 0.03983  loss_fcos_ctr: 0.5874  time: 1.2642  data_time: 0.0079  lr: 0.01  max_mem: 11783M
[08/24 05:09:19 d2.utils.events]:  eta: 21:12:40  iter: 41019  total_loss: 1.836  loss_mask: 0.3405  loss_maskiou: 0.004629  loss_fcos_cls: 0.5176  loss_fcos_loc: 0.3367  loss_fcos_ctr: 0.6379  time: 1.2641  data_time: 0.0077  lr: 0.01  max_mem: 11783M
[08/24 05:09:43 d2.utils.events]:  eta: 21:07:21  iter: 41039  total_loss: 1.638  loss_mas